In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2018-07-22 21:30:32.219664


In [2]:
secret = False #Will be put to True for a CASD export

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }<\style>"))

In [4]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle

In [5]:
pickle_year_variable_path = "C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/year_variable/"
year =  pickle.load(open(pickle_year_variable_path+"year_variable.p",'rb')); print(year)

2011


In [6]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
result_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/{}/'.format(year)
Store = pd.HDFStore(store_path)

In [7]:
Store

<class 'pandas.io.pytables.HDFStore'>
File path: C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5

In [8]:
# for key in Store.keys(): print(key)
# Store.close()

In [9]:
Store.close()
fisc_revdet = pd.read_hdf( store_path,"FISC_REVDET_2012")
fisc_revenu = pd.read_hdf(store_path,"FISC_REVENU_2012").sort_values("ID_FISC_LOG_DIFF")
fisc_individu = pd.read_hdf( store_path,"FISC_INDIVIDU_2012" ).sort_values("ID_FISC_LOG_DIFF")

In [10]:
to_load_id_log = ["couple_child_detached_or_attached_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_maj_ratach_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_child_ss_maj_id_fisc_log_{}.p".format(year),
                  "couple_mar_pac_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ss_enf_id_fisc_log_{}.p".format(year),
                  "celib_ac_enf_ss_maj_id_fisc_log_{}.p".format(year),
                  "celib_with_detached_child_id_fisc_log_{}.p".format(year),
                  "single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log_{}.p".format(year),
                  "concubin_with_or_without_children_id_fisc_log_{}.p".format(year),]

In [11]:
#Charge à la fois dans l'environnement de travail et dans un dictionnaire...
pickle_dict = dict()
for pickle_to_load in to_load_id_log:
    loaded_pickle =  pickle.load(open(pickles_path+'{}'.format(pickle_to_load),'rb') )
    globals()[pickle_to_load[:-2]] = loaded_pickle
    print(pickle_to_load, "len:" , len(globals()[pickle_to_load[:-2]]));
    pickle_dict[pickle_to_load[:-2]] = loaded_pickle

couple_child_detached_or_attached_id_fisc_log_2011.p len: 209913
couple_mar_pac_child_maj_ratach_id_fisc_log_2011.p len: 101089
couple_mar_pac_child_ss_maj_id_fisc_log_2011.p len: 535703
couple_mar_pac_ss_enf_id_fisc_log_2011.p len: 424582
celib_ss_enf_id_fisc_log_2011.p len: 171821
celib_ac_enf_ss_maj_id_fisc_log_2011.p len: 47080
celib_with_detached_child_id_fisc_log_2011.p len: 97738
single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log_2011.p len: 5157
concubin_with_or_without_children_id_fisc_log_2011.p len: 293475


### Load or create typfisc sumed over households

In [12]:
%%time
try : dependent_groupby_TYPE_FISC = pd.read_hdf(pickles_path+"/to_trash.h5", "groupby_fisc_individu_TYPE_FISC_{}".format(year))
except: 
    dependent_groupby_TYPE_FISC = fisc_individu.groupby('ID_FISC_LOG_DIFF').TYPE_FISC.apply("".join)
    dependent_groupby_TYPE_FISC.to_hdf(pickles_path+"/to_trash.h5", "groupby_fisc_individu_TYPE_FISC_{}".format(year))

Wall time: 338 ms


### Allocate household type to fiscr

In [13]:
%%time
### Create boolean filter
for id_log_temp in to_load_id_log:
    print(id_log_temp)
    print(id_log_temp[:-19])
    loaded_id_log = globals()[id_log_temp[:-2]]
    bool_filter = fisc_revenu.ID_FISC_LOG_DIFF.isin(loaded_id_log)
    fisc_revenu.loc[bool_filter, "Type_menage"] = id_log_temp[:-19]
fisc_revenu["Type_menage"] = fisc_revenu.Type_menage.astype('category')

couple_child_detached_or_attached_id_fisc_log_2011.p
couple_child_detached_or_attached
couple_mar_pac_child_maj_ratach_id_fisc_log_2011.p
couple_mar_pac_child_maj_ratach
couple_mar_pac_child_ss_maj_id_fisc_log_2011.p
couple_mar_pac_child_ss_maj
couple_mar_pac_ss_enf_id_fisc_log_2011.p
couple_mar_pac_ss_enf
celib_ss_enf_id_fisc_log_2011.p
celib_ss_enf
celib_ac_enf_ss_maj_id_fisc_log_2011.p
celib_ac_enf_ss_maj
celib_with_detached_child_id_fisc_log_2011.p
celib_with_detached_child
single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log_2011.p
single_ac_single_ac_enf_ac_maj_ratach
concubin_with_or_without_children_id_fisc_log_2011.p
concubin_with_or_without_children
Wall time: 7.08 s


In [14]:
fisc_revenu.Type_menage.value_counts(normalize = True, dropna=False)

couple_mar_pac_child_ss_maj              0.225071
NaN                                      0.207381
couple_mar_pac_ss_enf                    0.178384
concubin_with_or_without_children        0.123301
couple_child_detached_or_attached        0.088193
celib_ss_enf                             0.072189
couple_mar_pac_child_maj_ratach          0.042472
celib_with_detached_child                0.041064
celib_ac_enf_ss_maj                      0.019780
single_ac_single_ac_enf_ac_maj_ratach    0.002167
Name: Type_menage, dtype: float64

In [15]:
fisc_revenu["Type_menage"].value_counts(dropna=False)

couple_mar_pac_child_ss_maj              535703
NaN                                      493599
couple_mar_pac_ss_enf                    424581
concubin_with_or_without_children        293475
couple_child_detached_or_attached        209913
celib_ss_enf                             171821
couple_mar_pac_child_maj_ratach          101089
celib_with_detached_child                 97738
celib_ac_enf_ss_maj                       47080
single_ac_single_ac_enf_ac_maj_ratach      5157
Name: Type_menage, dtype: int64

In [16]:
## Reconstruct family structure: number of dependent children, number of minor children, number of major attached children,
# number of major detached childre.

In [17]:
dependent_groupby_TYPE_FISC.name = "Type_fisc_sum_over_fisc_log"
dependent_groupby_TYPE_FISC = dependent_groupby_TYPE_FISC.to_frame()

In [18]:
dependent_groupby_TYPE_FISC["nb_of_declarant_one_by_id_fisc_log"] = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.count('1')
dependent_groupby_TYPE_FISC["declarant_two_in_id_fisc_log"]  = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.contains('2')

In [19]:
df = dependent_groupby_TYPE_FISC

In [20]:
dependent_groupby_TYPE_FISC["nb_of_declarant_one_by_id_fisc_log"] = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.count('1')
dependent_groupby_TYPE_FISC["declarant_two_in_id_fisc_log"]  = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.contains('2')
dependent_groupby_TYPE_FISC["is_concubin_household"] = dependent_groupby_TYPE_FISC.index.isin(
                                                                    pickle_dict["concubin_with_or_without_children_id_fisc_log_{}".format(year)])
dependent_groupby_TYPE_FISC["is_celib_with_detached_child"] = dependent_groupby_TYPE_FISC.index.isin(
pickle_dict["celib_with_detached_child_id_fisc_log_{}".format(year)])
dependent_groupby_TYPE_FISC["Nb_attached_minor_child"] = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.count("[A-Z]")
dependent_groupby_TYPE_FISC["Nb_attached_major_child"]  = dependent_groupby_TYPE_FISC.Type_fisc_sum_over_fisc_log.str.count("[3-9]")

In [21]:
dependent_groupby_TYPE_FISC["is_concubin"] = dependent_groupby_TYPE_FISC.index.isin(pickle_dict["concubin_with_or_without_children_id_fisc_log_{}".format(year)])
dependent_groupby_TYPE_FISC["is_celib_with_detached_child"] = df.nb_of_declarant_one_by_id_fisc_log.index.isin(
                                                                                                pickle_dict["celib_with_detached_child_id_fisc_log_{}".format(year)])


In [22]:
dependent_groupby_TYPE_FISC.Nb_attached_major_child.value_counts()

0    2222905
1     149649
2      35586
3       4005
4        583
6         26
5          6
8          4
Name: Nb_attached_major_child, dtype: int64

In [23]:
dependent_groupby_TYPE_FISC["Nb_detached_major_child"] = (((df.nb_of_declarant_one_by_id_fisc_log-2) * df.is_concubin_household) 
                         + ((df.nb_of_declarant_one_by_id_fisc_log-1) * df.is_celib_with_detached_child)
                         + (df.declarant_two_in_id_fisc_log * (df.nb_of_declarant_one_by_id_fisc_log-1))
                        )

In [24]:
fisc_revenu["Nb_parents"] = ( fisc_revenu.Type_menage.isin(["couple_mar_pac_child_ss_maj",
                                  "couple_child_detached_or_attached",
                                  "couple_mar_pac_ss_enf", "couple_mar_pac_child_maj_ratach",
                                  "concubin_with_or_without_children"]) * 2
                            + fisc_revenu.Type_menage.isin(["celib_ss_enf",
                                                          "celib_with_detached_child",
                                                          "celib_ac_enf_ss_maj",]) * 1)

In [25]:
merge = pd.merge(fisc_revenu.reset_index(),dependent_groupby_TYPE_FISC.reset_index(), on = 'ID_FISC_LOG_DIFF' )

In [26]:
merge["Nb_child"] = merge.Nb_detached_major_child + merge.Nb_attached_major_child + merge.Nb_attached_minor_child

In [27]:
(merge.Nb_child +merge.Nb_parents == merge.NBPERSM).value_counts(normalize = True)

True     0.793792
False    0.206208
dtype: float64

In [28]:
(merge.Nb_child +merge.Nb_parents == merge.NBPERSM).value_counts(normalize = False)

True     1889349
False     490807
dtype: int64

In [29]:
### On vire les nans

In [30]:
merge = merge[merge.Type_menage.notnull()]

In [31]:
(merge.Nb_child +merge.Nb_parents == merge.NBPERSM).value_counts(normalize = False)

True     1881392
False       5165
dtype: int64

In [32]:
# 8 annoamalies TODO: voir pourquoi...

In [33]:
merge= merge[(merge.Nb_child +merge.Nb_parents == merge.NBPERSM)]

In [34]:
merge = merge.convert_objects()

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [35]:
#result_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [36]:
#merge.to_hdf(result_store_path, "step_2", format = "t")

In [37]:
#pd.read_hdf(result_store_path, "step_2")

In [ ]:
merge.head()

In [39]:
result_store_path

'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [40]:
child_or_parent = pd.read_hdf(result_store_path,"fisci_child_or_parent_{}".format(year))

In [41]:
child_or_parent["ID_FISC_FOY_DIFF"] =child_or_parent["ID_FISC_FOY_DIFF_y"]

In [ ]:
child_or_parent.T

In [43]:
hello = pd.merge(child_or_parent, fisc_individu, on = ["ID_FISC_FOY_DIFF","ORDREFIP","TYPE_FISC"] )

In [44]:
hello2 = pd.merge(hello, merge, on = ["ID_FISC_LOG_DIFF"], how = "left")

### Determine if child or parent

In [45]:
fisc_individu2 = pd.merge(merge[["Type_menage", "ID_FISC_LOG_DIFF"]], fisc_individu, on = ["ID_FISC_LOG_DIFF"], how = "right")

In [46]:
#On va dire que les deux plus vieux du ménage sont les parents

In [47]:
fisc_individu2 = fisc_individu2.sort_values("ANAIS", ascending=False) #On sort pour avoir les plus vieux en premier
fisc_individu2["Age_rank_in_household"] = fisc_individu2.groupby("ID_FISC_LOG_DIFF").cumcount() + 1

In [48]:
fisc_individu2["Anais_2nd_Declarant"] = fisc_individu[fisc_individu2.TYPE_FISC=="2"].ANAIS
grpby_anais_2nd_declarant = fisc_individu2[["ID_FISC_LOG_DIFF", "Anais_2nd_Declarant"]].groupby("ID_FISC_LOG_DIFF").Anais_2nd_Declarant.sum()
fisc_individu2.drop(columns = "Anais_2nd_Declarant", inplace = True)
fisc_individu2 = pd.merge(fisc_individu2, grpby_anais_2nd_declarant.reset_index(), how = "left", on = "ID_FISC_LOG_DIFF" ).sort_values("ID_FISC_LOG_DIFF")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [ ]:
grpby_anais_2nd_declarant

In [50]:
#TODO remplacer par += pour vérifier que personne n'est égal à 2 !

In [51]:
fisc_individu2["Parent"] = False
fisc_individu2["Child"] = False

In [52]:
#Parents
is_declarant = (fisc_individu2.TYPE_FISC.str.contains("[1-2]", regex=True)) #True only for 2 ! Make difference in age from the second declarant >= 18
is_less_than_18_yrs_younger_than_2nd_declarant = fisc_individu2.Anais_2nd_Declarant >= fisc_individu2.ANAIS +18
is_not_child = is_less_than_18_yrs_younger_than_2nd_declarant


fisc_individu2.loc[((fisc_individu2.Type_menage == "couple_mar_pac_ss_enf")&is_not_child), 'Parent' ] = True
fisc_individu2.loc[
    ((fisc_individu2.Type_menage == "couple_mar_pac_child_ss_maj")& is_declarant & is_not_child), 'Parent' ] += True
fisc_individu2.loc[
    ((fisc_individu2.Type_menage == "couple_mar_pac_child_maj_ratach")& is_declarant & is_not_child), 'Parent' ] += True
fisc_individu2.loc[
    ((fisc_individu2.Type_menage == "couple_child_detached_or_attached")& is_declarant & is_not_child), 'Parent' ] += True

fisc_individu2.loc[((fisc_individu2.Type_menage == "celib_ss_enf")&is_not_child), 'Parent' ] += True
fisc_individu2.loc[((fisc_individu2.Type_menage == "celib_ac_enf_ss_maj")&is_not_child), 'Parent' ] += True
fisc_individu2.loc[
    ((fisc_individu2.Type_menage == "single_ac_single_ac_enf_ac_maj_ratach")
     &(fisc_individu2.TYPE_FISC.str.contains("[1]", regex=True))&is_not_child),
                 'Parent' ] += True



C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [53]:
fisc_individu2.Parent.value_counts(dropna= False)

False    6196508
True     1325326
Name: Parent, dtype: int64

In [ ]:
### Mauvais merge !!! 

In [ ]:
fisc_individu2[["ID_FISC_LOG_DIFF","Type_menage","TYPE_FISC",
                "SEXE","JNAIS","MNAIS","ANAIS","Parent", "Anais_2nd_Declarant" ]]

In [55]:
fisc_individu2.Parent.value_counts(dropna= False)

False    6196508
True     1325326
Name: Parent, dtype: int64

In [56]:
# Enfants
is_major_attached = (fisc_individu2.TYPE_FISC.str.contains("[3-9]", regex=True))
is_minor_attached = (fisc_individu2.TYPE_FISC.str.contains("[A-K]", regex=True))
#Parents
fisc_individu2.loc[fisc_individu2.Type_menage == "couple_mar_pac_ss_enf", 'Child' ] += True
fisc_individu2.loc[(
    (fisc_individu2.Type_menage == "couple_mar_pac_child_ss_maj")
     & is_minor_attached ),'Child' ] += True



fisc_individu2.loc[
    ((fisc_individu2.Type_menage == "couple_mar_pac_child_maj_ratach") &(is_major_attached | is_minor_attached) ), 'Child' ] += True # Do not 
# fisc_individu2.loc[
#     ((fisc_individu2.Type_menage == "couple_child_detached_or_attached")& is_declarant), 'Parent' ] += True

# fisc_individu2.loc[fisc_individu2.Type_menage == "celib_ss_enf", 'Parent' ] += True
# fisc_individu2.loc[fisc_individu2.Type_menage == "celib_ac_enf_ss_maj", 'Parent' ] += True
# fisc_individu2.loc[
#     ((fisc_individu2.Type_menage == "single_ac_single_ac_enf_ac_maj_ratach")
#      &(fisc_individu2.TYPE_FISC.str.contains("[1]", regex=True))),
#                  'Parent' ] += True


C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\pandas\core\computation\expressions.py:179: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [57]:
fisc_individu2.Type_menage.value_counts()

couple_mar_pac_child_ss_maj              2215930
concubin_with_or_without_children         968106
couple_mar_pac_ss_enf                     849162
couple_child_detached_or_attached         830255
couple_mar_pac_child_maj_ratach           416606
celib_with_detached_child                 275268
celib_ss_enf                              171821
celib_ac_enf_ss_maj                       122458
single_ac_single_ac_enf_ac_maj_ratach          0
Name: Type_menage, dtype: int64

In [58]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)

now:  2018-07-22 21:33:47.139916
execution_time:  0:03:14.920252
